In [1]:
import numpy as np
import tensorflow as tf
#from tensorflow.keras.layers import TextVectorization
import pickle

2025-06-03 07:07:17.731473: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748934438.188064      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748934438.305308      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
def load_from_disk(path):

    with open(path, "rb") as f:
        loaded_list = pickle.load(f)

    return loaded_list

In [3]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

In [4]:
loaded_img_paths=load_from_disk('/kaggle/input/dev-images/dev_image_paths.pkl')

In [5]:
def load_and_preprocess_image(path):
    path = tf.strings.join(['/kaggle/input/flickr8k/Images/', path])
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize_with_pad(img, 224, 224)
    img = tf.cast(img, tf.float32)
    img = preprocess_input(img)  # Normalize for ResNet-50
    return img

In [6]:
data_encoder_set=tf.data.Dataset.from_tensor_slices(loaded_img_paths) \
.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE) \
.batch(16)

I0000 00:00:1748934482.484273      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1748934482.484976      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [7]:
# strategy = tf.distribute.MirroredStrategy()
# with strategy.scope():
encoder=ResNet50(include_top=False,
    weights='imagenet',
    input_shape=(224,224,3))
    

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [8]:
import numpy as np

results = []

for batch in data_encoder_set:
    # Forward pass
    pred = encoder(batch, training=False)  # use model() instead of .predict()
    results.append(pred.numpy())           # move to CPU as NumPy

# Concatenate all results on CPU
output_result = np.concatenate(results, axis=0)

print("Final output shape:", output_result.shape)


I0000 00:00:1748934508.049569      35 cuda_dnn.cc:529] Loaded cuDNN version 90300


Final output shape: (6070, 7, 7, 2048)


In [9]:
output_result[1]

array([[[0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ]],

       [[0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [1.4971554 , 0.        , 0.24541551, ..., 0.        ,
         0.        , 0.        ],
        [0.16220625, 0.        , 0.71257365, ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.        , 0.        , 1.295076  , ..., 0.847296  ,
         0.        , 0.        ],
        [0. 

In [10]:
np.save('/kaggle/working/processed_dev_images.npy',output_result)
print("Saved")

Saved


In [12]:
# output_result=encoder.predict(data_encoder_set)